- 先新增一個今天用的專案
- 綁定付費帳號
- enable Cloud Text-to-Speech API
- enable vertex ai api

在開始之前，你可能還需要知道一些資訊，[聲音清單](https://cloud.google.com/text-to-speech/docs/voices)，以及 AI 到底在文字轉語音上有什麼[成效](https://cloud.google.com/text-to-speech?hl=en#demo)

下載套件，這邊的錯誤不影響工作坊

In [1]:
%pip install --user --upgrade -q google-cloud-aiplatform google-cloud-texttospeech pydub pandas tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 16.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

沒有特別理由的話，只需要提供你的專案 ID

In [45]:

PROJECT_ID = "bwa-sample"  # @param {type:"string"}
TTS_LOCATION = "us"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}
API_ENV = "aiplatform.googleapis.com"  # @param {type:"string"}

In [50]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

    ! gcloud config set project {PROJECT_ID}
    ! gcloud auth application-default login -q

Updated property [core/project].
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=b5znzfFQfnhVAGfeGcmNFYB2NKA0AL&prompt=consent&token_usage=remote&access_type=offline&code_challenge=w4q3NuBeWcfjEnE6coZadLnRBee90elL4NgDily4yl0&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ATx3LY5yP_v9Z4oCfF797uUGrGWOC6p6jUr4jNa1VjBsYOR0fLxElF2H9HeRYC2a-vTd2Q

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Applicat

start

In [51]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    api_endpoint=f"{REGION}-{API_ENV}",
)
model = GenerativeModel(
  "gemini-1.5-flash-001",
  system_instruction=["你是一名藝術史專家，你所有的對話都會使用中文，並且你會盡可能詳細的回答"]
)

In [52]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

將圖片上傳到側邊的資料夾

In [53]:
import base64
filename = "The_Red_Boy.jpg"  # @param {type:"string"}


In [54]:
expect_output_format="""
期望格式：
```
Characters:

Thomas Lawrence
John Lambton

Thomas Lawrence : 畫作需要五個小時才能完成，議員你要和孩子一起在這邊等待嗎
John Lambton : 沒事的，對貴族來說，等待也是耐心的表現
...
```
"""
user_input_prompt = "我會提供這個畫作的背景故事，請你扮演裡面的角色，以角色對話的方式重現這則故事，請遵循背景故事得真實性和時間順序，提供一則長篇的中文故事以重現故事情況，對話應該要自然且符合情緒  背景故事：這張畫作的作者是 Thomas Lawrence，是在 John Lambton 議員在 1825 要求繪製他七歲的孩子 Charles William Lambton 的，當初 Lawrence 將男孩的衣服畫成了黃色，雖然男孩的父親被稱為「黃色花花公子」。然而， John Lambton 對此感到不滿，並強烈要求勞倫斯將衣服的顏色改成深紅色。  角色清單： - Thomas Lawrence 畫家 - John Lambton 議員 " # @param {type:"string"}
prompt = user_input_prompt + expect_output_format

print(prompt)

我會提供這個畫作的背景故事，請你扮演裡面的角色，以角色對話的方式重現這則故事，請遵循背景故事得真實性和時間順序，提供一則長篇的中文故事以重現故事情況，對話應該要自然且符合情緒  背景故事：這張畫作的作者是 Thomas Lawrence，是在 John Lambton 議員在 1825 要求繪製他七歲的孩子 Charles William Lambton 的，當初 Lawrence 將男孩的衣服畫成了黃色，雖然男孩的父親被稱為「黃色花花公子」。然而， John Lambton 對此感到不滿，並強烈要求勞倫斯將衣服的顏色改成深紅色。  角色清單： - Thomas Lawrence 畫家 - John Lambton 議員 
期望格式：
```
Characters:

Thomas Lawrence
John Lambton

Thomas Lawrence : 畫作需要五個小時才能完成，議員你要和孩子一起在這邊等待嗎
John Lambton : 沒事的，對貴族來說，等待也是耐心的表現
...
```



generate story

In [55]:
from vertexai.generative_models import Image
image_part = Part.from_image(Image.load_from_file(filename))

將圖片和文字，作為 API 參數，一起傳給多模態的模型

In [56]:
response = model.generate_content([image_part, prompt], generation_config=generation_config)

In [57]:
print(response.candidates[0].content.parts[0].text)

Characters:

Thomas Lawrence
John Lambton

Thomas Lawrence : 議員，這幅畫作需要五個小時才能完成，你要和孩子一起在這邊等待嗎？還是您希望先去別的地方休息一下呢？
John Lambton : 沒事的，對貴族來說，等待也是耐心的表現，尤其是等待能將自己孩子的英姿記錄在畫布上。說起來，勞倫斯先生，您可否將小查爾斯衣服的顏色改成深紅色？
Thomas Lawrence : 深紅色？議員，我以為您會喜歡我給小查爾斯搭配的黃色，這是一種充滿活力與朝氣的顏色，不是嗎？
John Lambton : 勞倫斯先生，您誤會了。我當然知道您是要表現出小查爾斯的天真爛漫，但這個黃色，實在是太像「黃色花花公子」了！您知道，我可不希望小查爾斯被世人誤認為是浪蕩子弟，我希望他成為一個正直、勇敢、有責任感的紳士，深紅色，才是更能體現這種氣質的顏色！
Thomas Lawrence : 議員，您說的沒錯。深紅色，的確更有力量，也更能表現出小查爾斯的堅毅和高貴。我會盡力將深紅色融入到畫作中，讓小查爾斯看起來更像一位真正的紳士。
John Lambton : 非常感謝您，勞倫斯先生。我相信您一定能將小查爾斯描繪得栩栩如生，他一定會喜歡這幅畫作的！
Thomas Lawrence : 議員，我一定會竭盡全力，將小查爾斯的英姿展現在世人面前。
John Lambton : 勞倫斯先生，您真是位出色的畫家，我對您的才華感到十分敬佩。
Thomas Lawrence : 議員過獎了，我不過是忠於自己的藝術追求罷了。
John Lambton : 我相信，您一定會在藝術界取得更高的成就，成為一名令人尊敬的藝術大師！
Thomas Lawrence : 謝謝您，議員，我會繼續努力，不辜負您的期望。
John Lambton : 勞倫斯先生，我對於這幅畫作充滿期待，我相信它將會成為我們家族的珍貴傳家寶！
Thomas Lawrence : 議員，您放心，我會盡心盡力，完成您心目中的理想作品。
John Lambton : 那麼，勞倫斯先生，我們就靜待佳音吧！
Thomas Lawrence : 好，議員，您請稍等，我會在五個小時後，將完成的畫作呈獻給您。


save story as .txt file

In [58]:
# todo save txt
story_file_name = "the_red_boy.txt"  # @param {type:"string"}
with open(story_file_name, 'w') as writefile:
    writefile.write(response.candidates[0].content.parts[0].text)

準備音檔

In [59]:
from IPython.display import Audio

import json
import os
from pathlib import Path
from typing import Dict, List, Tuple

from pydub import AudioSegment
from tqdm import tqdm
import pandas as pd

from google.api_core.client_options import ClientOptions
from google.cloud import texttospeech_v1beta1 as texttospeech
from vertexai.preview.generative_models import GenerativeModel, GenerationConfig

指定語音語言為繁體中文

In [60]:
DEFAULT_LANGUAGE = "cmn-TW"
# Voice used for narration, scene details, etc.
DEFAULT_VOICE = "cmn-TW-Standard-A"

tts_client = texttospeech.TextToSpeechClient(
    client_options=ClientOptions(
        api_endpoint=f"{TTS_LOCATION}-texttospeech.googleapis.com"
    )
)
# model = GenerativeModel("gemini-pro")

SILENCE_LENGTH = 200  # In Milliseconds
TXT_EXTENSION = ".txt"

helper function

In [61]:
def list_voices(
    language_code: str = DEFAULT_LANGUAGE, voice_type: str = "Standard"
) -> List[Dict]:
    response = tts_client.list_voices(language_code=language_code)
    # print(response.voices[0].language_codes)
    return [
        {
            "name": voice.name,
            "gender": texttospeech.SsmlVoiceGender(voice.ssml_gender).name.lower(),
            "language_codes": voice.language_codes,
        }

        for voice in response.voices
        if voice_type in voice.name and voice.name != DEFAULT_VOICE
    ]


def create_character_map(characters: List[str], voices: List[str]) -> Dict[str, str]:
    sample_json = """sample.json
```
{
  "Character1": "Voice Name",
  "Character2": "Voice Name",
  "Character3": "Voice Name"
}
```"""

    responses = model.generate_content(
        f"""Your job is to uniquely and appropriately match character names to voices available with Google Cloud Text to Speech.

The following is a list of available voices for Google Cloud Text to Speech in a JSON list.

{voices}

The following is a list of character names in a JSON list:

{characters}

Output a JSON formatted object mapping Character Names to Voice Names and nothing else:
{sample_json}

DO NOT INCLUDE text not json
""",
        generation_config=generation_config,
        stream=True,
    )
    json_string = ""
    # print(responses)
    for response in responses:
        json_string += response.text.replace("`", "").replace("json", "")

    print(json_string)
    return json.loads(json_string)


def synthesize_text(
    text: str, output: str, voice_name: str, language_code: str = DEFAULT_LANGUAGE
):
    response = tts_client.synthesize_speech(
        input=texttospeech.SynthesisInput(text=text),
        voice=texttospeech.VoiceSelectionParams(
            language_code=language_code,
            name=voice_name,
        ),
        audio_config=texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3
        ),
    )

    # The response's audio_content is binary.
    with open(output, "wb") as f:
        f.write(response.audio_content)


def combine_audio_files(audio_files: List[str], filename: str) -> str:
    full_audio = AudioSegment.silent(duration=SILENCE_LENGTH)

    for file in audio_files:
        sound = AudioSegment.from_mp3(file)
        silence = AudioSegment.silent(duration=SILENCE_LENGTH)
        full_audio += sound + silence
        os.remove(file)

    outfile_name = f"{Path(filename).stem}-complete.mp3"
    full_audio.export(outfile_name, format="mp3")
    return outfile_name


def get_characters(input_file: str) -> List[str]:
    character_list = []
    with open(input_file, "r") as f:
        lines = f.readlines()

    start_line = 0

    for i in range(start_line + 2, len(lines)):
        if lines[i] == "\n":
            break
        character_list.append(lines[i].strip())
    return character_list


def parse_file(
    input_file: str, character_to_voice: Dict[str, Tuple[str, str]]
) -> List[str]:
    with open(input_file, "r") as f:
        lines = f.readlines()

    line_number = 1
    output_files = []
    filename = Path(input_file).stem

    for line in tqdm(lines, "Parsing input file"):
        split_line = line.strip().split(": ", 1)

        character = split_line[0]
        if not character:  # Skip blank lines
            continue

        voice = character_to_voice.get(character.strip())

        # print(character)
        # print(voice)
        if not voice:
            print(f"No voice found for character {character}")
            continue

        if len(split_line) <= 1:
            dialogue = split_line[0]
        elif "Scene" in split_line[0]:
            dialogue = f"{split_line[0]}, {split_line[1]}"
        else:
            dialogue = split_line[1]

        output_file = f"{filename}-{line_number}.mp3"
        output_files.append(output_file)
        synthesize_text(dialogue, output_file, voice[0], voice[1])
        line_number += 1

    return output_files


條列出所有可用的語音模型，注意！！繁體中文的 standard 只有兩個男性一個女性的模型，如果角色過多，可以參考英文的

In [62]:
all_voices = list_voices()
print(list_voices())
# print(pd.DataFrame(all_voices))

[{'name': 'cmn-TW-Standard-B', 'gender': 'male', 'language_codes': ['cmn-TW']}, {'name': 'cmn-TW-Standard-C', 'gender': 'male', 'language_codes': ['cmn-TW']}]


In [63]:
# input_file = "TheRedBoy.txt"  # @param {type:"string"}

character_list = get_characters(story_file_name)
print(character_list)
print(all_voices)
if len(character_list) > len(all_voices):
    print(f"Too many characters {len(character_list)}. Max {len(all_voices)}")

['Thomas Lawrence', 'John Lambton']
[{'name': 'cmn-TW-Standard-B', 'gender': 'male', 'language_codes': ['cmn-TW']}, {'name': 'cmn-TW-Standard-C', 'gender': 'male', 'language_codes': ['cmn-TW']}]


為角色定義聲音

In [64]:
character_to_voice = create_character_map(character_list, all_voices)


{
  "Thomas Lawrence": "cmn-TW-Standard-B",
  "John Lambton": "cmn-TW-Standard-C"
}



In [65]:
voice_to_language = {voice['name']: voice['language_codes'][0] for voice in all_voices}

# Create the new variable
character_to_voice_language = {
    character: [voice, voice_to_language[voice]]
    for character, voice in character_to_voice.items()
}

print(character_to_voice_language)

{'Thomas Lawrence': ['cmn-TW-Standard-B', 'cmn-TW'], 'John Lambton': ['cmn-TW-Standard-C', 'cmn-TW']}


以角色對話方式呈現的故事，會用角色名稱作為音檔的切割分段，因此執行後會變成好幾個音檔

In [66]:
output_files = parse_file(story_file_name, character_to_voice_language)

Parsing input file:  20%|██        | 4/20 [00:00<00:00, 24.90it/s]

No voice found for character Characters:


Parsing input file: 100%|██████████| 20/20 [00:03<00:00,  6.54it/s]


這時才將所有音檔結合起來變成一個

In [67]:
outfile_name = combine_audio_files(output_files, story_file_name)
print(f"Audio content written to file {outfile_name}")

Audio content written to file the_red_boy-complete.mp3


In [68]:
Audio(outfile_name)